In [6]:
import tensorflow as tf

In [3]:
tf.__version__

'1.5.0'

In [3]:
!pip install --upgrade pip

  Found existing installation: pip 19.0.3
    Uninstalling pip-19.0.3:
      Successfully uninstalled pip-19.0.3


Could not install packages due to an EnvironmentError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\JUNGWO~1\\AppData\\Local\\Temp\\pip-uninstall-jme9_aly\\pip.exe'
Consider using the `--user` option or check the permissions.



In [2]:
!pip install pyrouge

  Stored in directory: C:\Users\Jungwoo Lim\AppData\Local\pip\Cache\wheels\75\d3\0c\e5b04e15b6b87c42e980de3931d2686e14d36e045058983599
Successfully built pyrouge


You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
####################################
# Author: Shashi Narayan
# Date: September 2016
# Project: Document Summarization
# H2020 Summa Project
# Comments: Jan 2017
# Improved for Reinforcement Learning
####################################

"""
Document Summarization System
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
import random
import sys
import time

import numpy as np
import tensorflow as tf

from reward_utils import Reward_Generator
from data_utils import DataProcessor
from my_flags import FLAGS
from my_model import MY_Model

C:\Users\Jungwoo Lim\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
######################## Batch Testing a model on some dataset ############

def batch_predict_with_a_model(data, model, session=None):

  data_logits = []
  data_labels = []
  data_weights = []
  
  step = 1
  while (step * FLAGS.batch_size) <= len(data.fileindices):
    # Get batch data as Numpy Arrays : Without shuffling
    batch_docnames, batch_docs, batch_label, batch_weight, batch_oracle_multiple, batch_reward_multiple = data.get_batch(((step-1)*FLAGS.batch_size), (step * FLAGS.batch_size))
    batch_logits = session.run(model.logits, feed_dict={model.document_placeholder: batch_docs})
    
    data_logits.append(batch_logits)
    data_labels.append(batch_label)
    data_weights.append(batch_weight)
    
    # Increase step
    step += 1

  # Check if any data left
  if (len(data.fileindices) > ((step-1)*FLAGS.batch_size)):
    # Get last batch as Numpy Arrays
    batch_docnames, batch_docs, batch_label, batch_weight, batch_oracle_multiple, batch_reward_multiple = data.get_batch(((step-1)*FLAGS.batch_size), len(data.fileindices))
    batch_logits = session.run(model.logits, feed_dict={model.document_placeholder: batch_docs})
    
    data_logits.append(batch_logits)
    data_labels.append(batch_label)
    data_weights.append(batch_weight)
    # print(data_logits) 
    
  # Convert list to tensors
  data_logits = tf.concat(0, data_logits)
  data_lables = tf.concat(0, data_labels)
  data_weights = tf.concat(0, data_weights)
  # print(data_logits,data_lables,data_weights)
  return data_logits, data_lables, data_weights 

######################## Training Mode ###########################

def train():
  """
  Training Mode: Create a new model and train the network
  """

  # Training: use the tf default graph
  with tf.Graph().as_default() and tf.device(FLAGS.use_gpu):

    config = tf.ConfigProto(allow_soft_placement = True)
    
    # Start a session
    with tf.Session(config = config) as sess:
      
      ### Prepare data for training
      print("Prepare vocab dict and read pretrained word embeddings ...")
      vocab_dict, word_embedding_array = DataProcessor().prepare_vocab_embeddingdict()
      # vocab_dict contains _PAD and _UNK but not word_embedding_array

      print("Prepare training data ...")
      train_data = DataProcessor().prepare_news_data(vocab_dict, data_type="training")
      
      print("Prepare validation data ...")
      validation_data = DataProcessor().prepare_news_data(vocab_dict, data_type="validation")

      print("Prepare ROUGE reward generator ...")
      rouge_generator = Reward_Generator()

      # Create Model with various operations
      model = MY_Model(sess, len(vocab_dict)-2)
      
      # Start training with some pretrained model
      start_epoch = 1
      # selected_modelpath = FLAGS.train_dir+"/model.ckpt.epoch-"+str(start_epoch-1)
      # if not (os.path.isfile(selected_modelpath)):
      #   print("Model not found in checkpoint folder.")
      #   exit(0)
      # # Reload saved model and test
      # print("Reading model parameters from %s" % selected_modelpath)
      # model.saver.restore(sess, selected_modelpath)
      # print("Model loaded.")

      # Initialize word embedding before training
      print("Initialize word embedding vocabulary with pretrained embeddings ...")
      sess.run(model.vocab_embed_variable.assign(word_embedding_array))

      ########### Start (No Mixer) Training : Reinforcement learning ################
      # Reward aware training as part of Reward weighted CE ,  
      # No Curriculam learning: No annealing, consider full document like in MRT
      # Multiple Samples (include gold sample),  No future reward, Similar to MRT
      # During training does not use PYROUGE to avoid multiple file rewritings
      # Approximate MRT with multiple pre-estimated oracle samples
      # June 2017: Use Single sample from multiple oracles
      ###############################################################################

      print("Start Reinforcement Training (single rollout from largest prob mass) ...")

      for epoch in range(start_epoch, FLAGS.train_epoch_wce + 1):
        print("MRT: Epoch "+str(epoch))
        
        print("MRT: Epoch "+str(epoch)+" : Reshuffle training document indices")
        train_data.shuffle_fileindices()
          
        print("MRT: Epoch "+str(epoch)+" : Restore Rouge Dict")
        rouge_generator.restore_rouge_dict()
          
        # Start Batch Training
        step = 1
        while (step * FLAGS.batch_size) <= len(train_data.fileindices):
          # Get batch data as Numpy Arrays
          batch_docnames, batch_docs, batch_label, batch_weight, batch_oracle_multiple, batch_reward_multiple = train_data.get_batch(((step-1)*FLAGS.batch_size), 
                                                                                                                                     (step * FLAGS.batch_size))
          # print(batch_docnames)
          # print(batch_label[0])
          # print(batch_weight[0])
          # print(batch_oracle_multiple[0])
          # print(batch_reward_multiple[0])
          # exit(0)
          
          # Print the progress 
          if (step % FLAGS.training_checkpoint)  == 0:
            
            ce_loss_val, ce_loss_sum, acc_val, acc_sum = sess.run([model.rewardweighted_cross_entropy_loss_multisample, model.rewardweighted_ce_multisample_loss_summary, 
                                                                   model.accuracy, model.taccuracy_summary],
                                                                  feed_dict={model.document_placeholder: batch_docs, 
                                                                             model.predicted_multisample_label_placeholder: batch_oracle_multiple, 
                                                                             model.actual_reward_multisample_placeholder: batch_reward_multiple,
                                                                             model.label_placeholder: batch_label,
                                                                             model.weight_placeholder: batch_weight})
            
            # Print Summary to Tensor Board
            model.summary_writer.add_summary(ce_loss_sum, ((epoch-1)*len(train_data.fileindices)+ step*FLAGS.batch_size))
            model.summary_writer.add_summary(acc_sum, ((epoch-1)*len(train_data.fileindices)+step*FLAGS.batch_size))

            print("MRT: Epoch "+str(epoch)+" : Covered " + str(step*FLAGS.batch_size)+"/"+str(len(train_data.fileindices)) + 
                  " : Minibatch Reward Weighted Multisample CE Loss= {:.6f}".format(ce_loss_val) + " : Minibatch training accuracy= {:.6f}".format(acc_val))

          # Run optimizer: optimize policy network 
          sess.run([model.train_op_policynet_expreward], feed_dict={model.document_placeholder: batch_docs, 
                                                                    model.predicted_multisample_label_placeholder: batch_oracle_multiple, 
                                                                    model.actual_reward_multisample_placeholder: batch_reward_multiple,
                                                                    model.weight_placeholder: batch_weight})
          
          # Increase step
          step += 1
          
          # if step == 20:
          #   break 

        # Save Model 
        print("MRT: Epoch "+str(epoch)+" : Saving model after epoch completion")
        checkpoint_path = os.path.join(FLAGS.train_dir, "model.ckpt.epoch-"+str(epoch))
        model.saver.save(sess, checkpoint_path)
        
        # Backup Rouge Dict
        print("MRT: Epoch "+str(epoch)+" : Saving rouge dictionary")
        rouge_generator.save_rouge_dict()
        
        # Performance on the validation set 
        print("MRT: Epoch "+str(epoch)+" : Performance on the validation data")
        # Get Predictions: Prohibit the use of gold labels
        validation_logits, validation_labels, validation_weights = batch_predict_with_a_model(validation_data, model, session=sess)
        # Validation Accuracy and Prediction
        validation_acc, validation_sum = sess.run([model.final_accuracy, model.vaccuracy_summary], feed_dict={model.logits_placeholder: validation_logits.eval(session=sess), 
                                                                                                              model.label_placeholder: validation_labels.eval(session=sess), 
                                                                                                              model.weight_placeholder: validation_weights.eval(session=sess)})
        # Print Validation Summary 
        model.summary_writer.add_summary(validation_sum, (epoch*len(train_data.fileindices)))
        
        print("MRT: Epoch "+str(epoch)+" : Validation ("+str(len(validation_data.fileindices))+") accuracy= {:.6f}".format(validation_acc))
        # Writing validation predictions and final summaries
        print("MRT: Epoch "+str(epoch)+" : Writing final validation summaries")
        validation_data.write_prediction_summaries(validation_logits, "model.ckpt.epoch-"+str(epoch), session=sess)
        # Extimate Rouge Scores
        rouge_score = rouge_generator.get_full_rouge(FLAGS.train_dir+"/model.ckpt.epoch-"+str(epoch)+".validation-summary-topranked", "validation")
        print("MRT: Epoch "+str(epoch)+" : Validation ("+str(len(validation_data.fileindices))+") rouge= {:.6f}".format(rouge_score))
        
        # break

      print("Optimization Finished!")

# ######################## Test Mode ###########################

def test():
  """
  Test Mode: Loads an existing model and test it on the test set
  """
  
  # Training: use the tf default graph

  with tf.Graph().as_default() and tf.device(FLAGS.use_gpu):

    config = tf.ConfigProto(allow_soft_placement = True)

    # Start a session
    with tf.Session(config = config) as sess:
           
      ### Prepare data for training
      print("Prepare vocab dict and read pretrained word embeddings ...")
      vocab_dict, word_embedding_array = DataProcessor().prepare_vocab_embeddingdict()
      # vocab_dict contains _PAD and _UNK but not word_embedding_array
      
      print("Prepare test data ...")
      test_data = DataProcessor().prepare_news_data(vocab_dict, data_type="test")

      # Create Model with various operations
      model = MY_Model(sess, len(vocab_dict)-2)

      # # Initialize word embedding before training
      # print("Initialize word embedding vocabulary with pretrained embeddings ...")
      # sess.run(model.vocab_embed_variable.assign(word_embedding_array))

      # Select the model
      if (os.path.isfile(FLAGS.train_dir+"/model.ckpt.epoch-"+str(FLAGS.model_to_load))):
        selected_modelpath = FLAGS.train_dir+"/model.ckpt.epoch-"+str(FLAGS.model_to_load)
      else:
        print("Model not found in checkpoint folder.")
        exit(0)
      
      # Reload saved model and test
      print("Reading model parameters from %s" % selected_modelpath)
      model.saver.restore(sess, selected_modelpath)
      print("Model loaded.")

      # Initialize word embedding before training
      print("Initialize word embedding vocabulary with pretrained embeddings ...")
      sess.run(model.vocab_embed_variable.assign(word_embedding_array))

      # Test Accuracy and Prediction
      print("Performance on the test data:")
      FLAGS.authorise_gold_label = False
      test_logits, test_labels, test_weights = batch_predict_with_a_model(test_data, model, session=sess)
      test_acc = sess.run(model.final_accuracy, feed_dict={model.logits_placeholder: test_logits.eval(session=sess), 
                                                             model.label_placeholder: test_labels.eval(session=sess), 
                                                             model.weight_placeholder: test_weights.eval(session=sess)})
      # Print Test Summary
      print("Test ("+str(len(test_data.fileindices))+") accuracy= {:.6f}".format(test_acc))
      # Writing test predictions and final summaries
      test_data.write_prediction_summaries(test_logits, "model.ckpt.epoch-"+str(FLAGS.model_to_load), session=sess)

######################## Main Function ###########################

def main(_):
  if FLAGS.exp_mode == "train":
    train()
  else:
    test()

if __name__ == "__main__":
  tf.app.run()

Prepare vocab dict and read pretrained word embeddings ...
Reading pretrained word embeddings file: C:\Users\Jungwoo Lim\Documents\GitHub\Refresh\address\data\1-billion-word-language-modeling-benchmark-r13output.word2vec.vec
0 ...
100000 ...
200000 ...
300000 ...
400000 ...
500000 ...
Read pretrained embeddings: (559183, 200)
Size of vocab: 559185 (_PAD:0, _UNK:1)
Writing vocab file: C:\Users\Jungwoo Lim\Documents\GitHub\Refresh\address\to\training\directory\vocab.txt
Prepare training data ...
Data file prefix (.doc, .title, .image, .label.multipleoracle): C:\Users\Jungwoo Lim\Documents\GitHub\Refresh\address\data\preprocessed-input-directory/cnn.training
Data sizes: 90262 90262 90262 90262
Reading data (no padding to save memory) ...
0 ...
10000 ...
20000 ...
30000 ...
40000 ...
50000 ...
60000 ...
70000 ...
80000 ...
90000 ...
Writing data files with prefix (.filename, .doc, .title, .image, .label, .weight, .rewards): C:\Users\Jungwoo Lim\Documents\GitHub\Refresh\address\to\training\

MRT: Epoch 1 : Covered 820/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.060068 : Minibatch training accuracy= 0.896907
MRT: Epoch 1 : Covered 840/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.245074 : Minibatch training accuracy= 0.873516
MRT: Epoch 1 : Covered 860/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.315844 : Minibatch training accuracy= 0.905887
MRT: Epoch 1 : Covered 880/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.106751 : Minibatch training accuracy= 0.876988
MRT: Epoch 1 : Covered 900/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.505981 : Minibatch training accuracy= 0.885550
MRT: Epoch 1 : Covered 920/90262 : Minibatch Reward Weighted Multisample CE Loss= 2.924567 : Minibatch training accuracy= 0.892563
MRT: Epoch 1 : Covered 940/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.121534 : Minibatch training accuracy= 0.879473
MRT: Epoch 1 : Covered 960/90262 : Minibatch Reward Weighted Multisample CE Loss= 4

MRT: Epoch 1 : Covered 2080/90262 : Minibatch Reward Weighted Multisample CE Loss= 2.984259 : Minibatch training accuracy= 0.850583
MRT: Epoch 1 : Covered 2100/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.404133 : Minibatch training accuracy= 0.923166
MRT: Epoch 1 : Covered 2120/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.713287 : Minibatch training accuracy= 0.893626
MRT: Epoch 1 : Covered 2140/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.095722 : Minibatch training accuracy= 0.868665
MRT: Epoch 1 : Covered 2160/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.508728 : Minibatch training accuracy= 0.903095
MRT: Epoch 1 : Covered 2180/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.661273 : Minibatch training accuracy= 0.894910
MRT: Epoch 1 : Covered 2200/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.302888 : Minibatch training accuracy= 0.908405
MRT: Epoch 1 : Covered 2220/90262 : Minibatch Reward Weighted Multisample CE

MRT: Epoch 1 : Covered 3340/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.569903 : Minibatch training accuracy= 0.894344
MRT: Epoch 1 : Covered 3360/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.248807 : Minibatch training accuracy= 0.895206
MRT: Epoch 1 : Covered 3380/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.525803 : Minibatch training accuracy= 0.907406
MRT: Epoch 1 : Covered 3400/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.658669 : Minibatch training accuracy= 0.895932
MRT: Epoch 1 : Covered 3420/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.654850 : Minibatch training accuracy= 0.903198
MRT: Epoch 1 : Covered 3440/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.065576 : Minibatch training accuracy= 0.895583
MRT: Epoch 1 : Covered 3460/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.340832 : Minibatch training accuracy= 0.891745
MRT: Epoch 1 : Covered 3480/90262 : Minibatch Reward Weighted Multisample CE

MRT: Epoch 1 : Covered 4600/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.324103 : Minibatch training accuracy= 0.892758
MRT: Epoch 1 : Covered 4620/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.113213 : Minibatch training accuracy= 0.897056
MRT: Epoch 1 : Covered 4640/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.278108 : Minibatch training accuracy= 0.874340
MRT: Epoch 1 : Covered 4660/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.186037 : Minibatch training accuracy= 0.910586
MRT: Epoch 1 : Covered 4680/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.271154 : Minibatch training accuracy= 0.874293
MRT: Epoch 1 : Covered 4700/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.314471 : Minibatch training accuracy= 0.901464
MRT: Epoch 1 : Covered 4720/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.428080 : Minibatch training accuracy= 0.893723
MRT: Epoch 1 : Covered 4740/90262 : Minibatch Reward Weighted Multisample CE

...

MRT: Epoch 1 : Covered 23280/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.185078 : Minibatch training accuracy= 0.900075
MRT: Epoch 1 : Covered 23300/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.757254 : Minibatch training accuracy= 0.899100
MRT: Epoch 1 : Covered 23320/90262 : Minibatch Reward Weighted Multisample CE Loss= 2.844405 : Minibatch training accuracy= 0.887102
MRT: Epoch 1 : Covered 23340/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.175537 : Minibatch training accuracy= 0.907000
MRT: Epoch 1 : Covered 23360/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.289738 : Minibatch training accuracy= 0.894154
MRT: Epoch 1 : Covered 23380/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.075922 : Minibatch training accuracy= 0.906555
MRT: Epoch 1 : Covered 23400/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.732002 : Minibatch training accuracy= 0.936483
MRT: Epoch 1 : Covered 23420/90262 : Minibatch Reward Weighted Multis

MRT: Epoch 1 : Covered 24520/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.066102 : Minibatch training accuracy= 0.888594
MRT: Epoch 1 : Covered 24540/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.228658 : Minibatch training accuracy= 0.873832
MRT: Epoch 1 : Covered 24560/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.266701 : Minibatch training accuracy= 0.909087
MRT: Epoch 1 : Covered 24580/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.147434 : Minibatch training accuracy= 0.912129
MRT: Epoch 1 : Covered 24600/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.723759 : Minibatch training accuracy= 0.904617
MRT: Epoch 1 : Covered 24620/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.130467 : Minibatch training accuracy= 0.870530
MRT: Epoch 1 : Covered 24640/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.194736 : Minibatch training accuracy= 0.918144
MRT: Epoch 1 : Covered 24660/90262 : Minibatch Reward Weighted Multis

MRT: Epoch 1 : Covered 25760/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.019235 : Minibatch training accuracy= 0.872383
MRT: Epoch 1 : Covered 25780/90262 : Minibatch Reward Weighted Multisample CE Loss= 2.969896 : Minibatch training accuracy= 0.885509
MRT: Epoch 1 : Covered 25800/90262 : Minibatch Reward Weighted Multisample CE Loss= 2.871511 : Minibatch training accuracy= 0.910596
MRT: Epoch 1 : Covered 25820/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.138437 : Minibatch training accuracy= 0.871288
MRT: Epoch 1 : Covered 25840/90262 : Minibatch Reward Weighted Multisample CE Loss= 2.924735 : Minibatch training accuracy= 0.865510
MRT: Epoch 1 : Covered 25860/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.555611 : Minibatch training accuracy= 0.892417
MRT: Epoch 1 : Covered 25880/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.506713 : Minibatch training accuracy= 0.913986
MRT: Epoch 1 : Covered 25900/90262 : Minibatch Reward Weighted Multis

MRT: Epoch 1 : Covered 27000/90262 : Minibatch Reward Weighted Multisample CE Loss= 2.864309 : Minibatch training accuracy= 0.872321
MRT: Epoch 1 : Covered 27020/90262 : Minibatch Reward Weighted Multisample CE Loss= 2.201738 : Minibatch training accuracy= 0.892941
MRT: Epoch 1 : Covered 27040/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.228318 : Minibatch training accuracy= 0.886678
MRT: Epoch 1 : Covered 27060/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.893368 : Minibatch training accuracy= 0.873170
MRT: Epoch 1 : Covered 27080/90262 : Minibatch Reward Weighted Multisample CE Loss= 4.320212 : Minibatch training accuracy= 0.871565
MRT: Epoch 1 : Covered 27100/90262 : Minibatch Reward Weighted Multisample CE Loss= 4.163011 : Minibatch training accuracy= 0.863689
MRT: Epoch 1 : Covered 27120/90262 : Minibatch Reward Weighted Multisample CE Loss= 3.870142 : Minibatch training accuracy= 0.889648
MRT: Epoch 1 : Covered 27140/90262 : Minibatch Reward Weighted Multis

KeyboardInterrupt: 

In [5]:

######################## Batch Testing a model on some dataset ############

def batch_predict_with_a_model(data, model, session=None):

    data_logits = []
    data_labels = []
    data_weights = []
  
    step = 1
    while (step * FLAGS.batch_size) <= len(data.fileindices):
    # Get batch data as Numpy Arrays : Without shuffling
        batch_docnames, batch_docs, batch_label, batch_weight, batch_oracle_multiple, batch_reward_multiple = data.get_batch(((step-1)*FLAGS.batch_size), (step * FLAGS.batch_size))
        batch_logits = session.run(model.logits, feed_dict={model.document_placeholder: batch_docs})
    
        data_logits.append(batch_logits)
        data_labels.append(batch_label)
        data_weights.append(batch_weight)
    
        # Increase step
        step += 1

    # Check if any data left
    if (len(data.fileindices) > ((step-1)*FLAGS.batch_size)):
        # Get last batch as Numpy Arrays
        batch_docnames, batch_docs, batch_label, batch_weight, batch_oracle_multiple, batch_reward_multiple = data.get_batch(((step-1)*FLAGS.batch_size), len(data.fileindices))
        batch_logits = session.run(model.logits, feed_dict={model.document_placeholder: batch_docs})
    
        data_logits.append(batch_logits)
        data_labels.append(batch_label)
        data_weights.append(batch_weight)
        # print(data_logits) 
    
    # Convert list to tensors
    data_logits = tf.concat(0, data_logits)
    data_lables = tf.concat(0, data_labels)
    data_weights = tf.concat(0, data_weights)
    # print(data_logits,data_lables,data_weights)
    return data_logits, data_lables, data_weights 

######################## Training Mode ###########################

def train():
  """
  Training Mode: Create a new model and train the network
  """

    # Training: use the tf default graph
    with tf.Graph().as_default() and tf.device(FLAGS.use_gpu):

        config = tf.ConfigProto(allow_soft_placement = True)
    
        # Start a session
        with tf.Session(config = config) as sess:
      
            ### Prepare data for training
            print("Prepare vocab dict and read pretrained word embeddings ...")
            vocab_dict, word_embedding_array = DataProcessor().prepare_vocab_embeddingdict()
            # vocab_dict contains _PAD and _UNK but not word_embedding_array

            print("Prepare training data ...")
            train_data = DataProcessor().prepare_news_data(vocab_dict, data_type="training")
      
            print("Prepare validation data ...")
            validation_data = DataProcessor().prepare_news_data(vocab_dict, data_type="validation")

            print("Prepare ROUGE reward generator ...")
            rouge_generator = Reward_Generator()

        # Create Model with various operations
        model = MY_Model(sess, len(vocab_dict)-2)
      
        # Start training with some pretrained model
        start_epoch = 1
        # selected_modelpath = FLAGS.train_dir+"/model.ckpt.epoch-"+str(start_epoch-1)
        # if not (os.path.isfile(selected_modelpath)):
        #   print("Model not found in checkpoint folder.")
        #   exit(0)
        # # Reload saved model and test
        # print("Reading model parameters from %s" % selected_modelpath)
        # model.saver.restore(sess, selected_modelpath)
        # print("Model loaded.")

        # Initialize word embedding before training
        print("Initialize word embedding vocabulary with pretrained embeddings ...")
        sess.run(model.vocab_embed_variable.assign(word_embedding_array))

        ########### Start (No Mixer) Training : Reinforcement learning ################
        # Reward aware training as part of Reward weighted CE ,  
        # No Curriculam learning: No annealing, consider full document like in MRT
        # Multiple Samples (include gold sample),  No future reward, Similar to MRT
        # During training does not use PYROUGE to avoid multiple file rewritings
        # Approximate MRT with multiple pre-estimated oracle samples
        # June 2017: Use Single sample from multiple oracles
        ###############################################################################

        print("Start Reinforcement Training (single rollout from largest prob mass) ...")

        for epoch in range(start_epoch, FLAGS.train_epoch_wce + 1):
            print("MRT: Epoch "+str(epoch))
        
            print("MRT: Epoch "+str(epoch)+" : Reshuffle training document indices")
            train_data.shuffle_fileindices()
          
            print("MRT: Epoch "+str(epoch)+" : Restore Rouge Dict")
            rouge_generator.restore_rouge_dict()
          
            # Start Batch Training
            step = 1
            while (step * FLAGS.batch_size) <= len(train_data.fileindices):
                # Get batch data as Numpy Arrays
                batch_docnames, batch_docs, batch_label, batch_weight, batch_oracle_multiple, batch_reward_multiple = train_data.get_batch(((step-1)*FLAGS.batch_size), (step * FLAGS.batch_size))
                # print(batch_docnames)
                # print(batch_label[0])
                # print(batch_weight[0])
                # print(batch_oracle_multiple[0])
                # print(batch_reward_multiple[0])
                # exit(0)
          
                # Print the progress 
                if (step % FLAGS.training_checkpoint)  == 0:
                    ce_loss_val, ce_loss_sum, acc_val, acc_sum = sess.run([model.rewardweighted_cross_entropy_loss_multisample, model.rewardweighted_ce_multisample_loss_summary, model.accuracy, model.taccuracy_summary],
                                                                          feed_dict={model.document_placeholder: batch_docs, 
                                                                             model.predicted_multisample_label_placeholder: batch_oracle_multiple, 
                                                                             model.actual_reward_multisample_placeholder: batch_reward_multiple,
                                                                             model.label_placeholder: batch_label,
                                                                             model.weight_placeholder: batch_weight})
            
                    # Print Summary to Tensor Board
                    model.summary_writer.add_summary(ce_loss_sum, ((epoch-1)*len(train_data.fileindices)+ step*FLAGS.batch_size))
                    model.summary_writer.add_summary(acc_sum, ((epoch-1)*len(train_data.fileindices)+step*FLAGS.batch_size))

                    print("MRT: Epoch "+str(epoch)+" : Covered " + str(step*FLAGS.batch_size)+"/"+str(len(train_data.fileindices)) + 
                  " : Minibatch Reward Weighted Multisample CE Loss= {:.6f}".format(ce_loss_val) + " : Minibatch training accuracy= {:.6f}".format(acc_val))

                # Run optimizer: optimize policy network 
                sess.run([model.train_op_policynet_expreward], feed_dict={model.document_placeholder: batch_docs, 
                                                                    model.predicted_multisample_label_placeholder: batch_oracle_multiple, 
                                                                    model.actual_reward_multisample_placeholder: batch_reward_multiple,
                                                                    model.weight_placeholder: batch_weight})
          
                # Increase step
                step += 1
          
                # if step == 20:
                #   break 

            # Save Model 
            print("MRT: Epoch "+str(epoch)+" : Saving model after epoch completion")
            checkpoint_path = os.path.join(FLAGS.train_dir, "model.ckpt.epoch-"+str(epoch))
            model.saver.save(sess, checkpoint_path)
        
            # Backup Rouge Dict
            print("MRT: Epoch "+str(epoch)+" : Saving rouge dictionary")
            rouge_generator.save_rouge_dict()
        
            # Performance on the validation set 
            print("MRT: Epoch "+str(epoch)+" : Performance on the validation data")
            # Get Predictions: Prohibit the use of gold labels
            validation_logits, validation_labels, validation_weights = batch_predict_with_a_model(validation_data, model, session=sess)
            # Validation Accuracy and Prediction
            validation_acc, validation_sum = sess.run([model.final_accuracy, model.vaccuracy_summary], feed_dict={model.logits_placeholder: validation_logits.eval(session=sess), 
                                                                                                              model.label_placeholder: validation_labels.eval(session=sess), 
                                                                                                              model.weight_placeholder: validation_weights.eval(session=sess)})
            # Print Validation Summary 
            model.summary_writer.add_summary(validation_sum, (epoch*len(train_data.fileindices)))
        
            print("MRT: Epoch "+str(epoch)+" : Validation ("+str(len(validation_data.fileindices))+") accuracy= {:.6f}".format(validation_acc))
            # Writing validation predictions and final summaries
            print("MRT: Epoch "+str(epoch)+" : Writing final validation summaries")
            validation_data.write_prediction_summaries(validation_logits, "model.ckpt.epoch-"+str(epoch), session=sess)
            # Extimate Rouge Scores
            rouge_score = rouge_generator.get_full_rouge(FLAGS.train_dir+"/model.ckpt.epoch-"+str(epoch)+".validation-summary-topranked", "validation")
            print("MRT: Epoch "+str(epoch)+" : Validation ("+str(len(validation_data.fileindices))+") rouge= {:.6f}".format(rouge_score))
        
            # break

        print("Optimization Finished!")

# ######################## Test Mode ###########################

def test():
  """
  Test Mode: Loads an existing model and test it on the test set
  """
  
    # Training: use the tf default graph

    with tf.Graph().as_default() and tf.device(FLAGS.use_gpu):

        config = tf.ConfigProto(allow_soft_placement = True)

        # Start a session
        with tf.Session(config = config) as sess:
           
            ### Prepare data for training
            print("Prepare vocab dict and read pretrained word embeddings ...")
            vocab_dict, word_embedding_array = DataProcessor().prepare_vocab_embeddingdict()
            # vocab_dict contains _PAD and _UNK but not word_embedding_array
      
            print("Prepare test data ...")
            test_data = DataProcessor().prepare_news_data(vocab_dict, data_type="test")

            # Create Model with various operations
            model = MY_Model(sess, len(vocab_dict)-2)

            # # Initialize word embedding before training
            # print("Initialize word embedding vocabulary with pretrained embeddings ...")
            # sess.run(model.vocab_embed_variable.assign(word_embedding_array))

            # Select the model
            if (os.path.isfile(FLAGS.train_dir+"/model.ckpt.epoch-"+str(FLAGS.model_to_load))):
                selected_modelpath = FLAGS.train_dir+"/model.ckpt.epoch-"+str(FLAGS.model_to_load)
            else:
                print("Model not found in checkpoint folder.")
                exit(0)
      
            # Reload saved model and test
            print("Reading model parameters from %s" % selected_modelpath)
            model.saver.restore(sess, selected_modelpath)
            print("Model loaded.")

            # Initialize word embedding before training
            print("Initialize word embedding vocabulary with pretrained embeddings ...")
            sess.run(model.vocab_embed_variable.assign(word_embedding_array))

            # Test Accuracy and Prediction
            print("Performance on the test data:")
            FLAGS.authorise_gold_label = False
            test_logits, test_labels, test_weights = batch_predict_with_a_model(test_data, model, session=sess)
            test_acc = sess.run(model.final_accuracy, feed_dict={model.logits_placeholder: test_logits.eval(session=sess), 
                                                             model.label_placeholder: test_labels.eval(session=sess), 
                                                             model.weight_placeholder: test_weights.eval(session=sess)})
            # Print Test Summary
            print("Test ("+str(len(test_data.fileindices))+") accuracy= {:.6f}".format(test_acc))
            # Writing test predictions and final summaries
            test_data.write_prediction_summaries(test_logits, "model.ckpt.epoch-"+str(FLAGS.model_to_load), session=sess)

######################## Main Function ###########################

def main(_):
    if FLAGS.exp_mode == "train":
        train()
    else:
        test()

if __name__ == "__main__":
    tf.app.run()






IndentationError: unexpected indent (<ipython-input-5-e358579bd0c3>, line 49)